In [ ]:
# Clean uninstall of conflicting packages
!pip uninstall -y torch torchvision torchaudio transformers bertopic sentence-transformers

# Install compatible versions
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install transformers==4.41.0 sentence-transformers==2.6.1 --quiet
!pip install bertopic[all]==0.16.0 --quiet


In [25]:
import pandas as pd
import plotly.express as px
import os
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import umap
import hdbscan
import re

In [26]:
from google.colab import drive
drive.mount('/content/drive')
# Define paths
df1_dir = '/content/drive/MyDrive/casereports/defaultCR_selection.csv'
df2_dir = '/content/drive/MyDrive/casereports/relatedCR_selection.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/casereports/pybioc /content/

In [27]:
def load_embeddings(emb_file_path, paper_id_path):
    # --- Configuration (match your saving paths) ---
    OUTPUT_DIR = "paper_embeddings_output"
    EMBEDDINGS_FILE = os.path.join(OUTPUT_DIR, emb_file_path)
    PAPER_IDS_FILE = os.path.join(OUTPUT_DIR, paper_id_path)

    # --- Load Embeddings and IDs ---
    try:
        all_paper_embeddings = np.load(EMBEDDINGS_FILE)
        with open(PAPER_IDS_FILE, 'r', encoding='utf-8') as f:
            processed_paper_ids = [line.strip() for line in f]

        print(f"Loaded {len(processed_paper_ids)} embeddings with shape: {all_paper_embeddings.shape}")
        return all_paper_embeddings

    except FileNotFoundError:
        print(f"Error: Embeddings or IDs file not found. Please ensure '{OUTPUT_DIR}' exists and contains the files.")
        print("Make sure you've run the embedding generation script first.")
        exit()

In [28]:
def clean_research_paper_text(text):
    """
    Cleans raw text extracted from a research paper.
    Adapted from previous discussion.
    """
    if not text:
        return ""

    # 1. Remove initial quote mark if present
    text = text.lstrip("'")

    # 2. Replace escaped newlines/tabs with space and remove backslashes
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'\\t', ' ', text)
    text = re.sub(r'\\', '', text)

    # 3. Handle specific noisy patterns from tables and statistical outputs
    lines = text.split('\n')
    cleaned_lines = []
    table_pattern = re.compile(r'\t.*?\(\d+\.?\d*\).*?\[.*?\]|\tN/A|\t\d+\s*\(.*?\)|\t[A-Za-z]+, [A-Za-z]+, [A-Za-z]+')

    for line in lines:
        if re.search(r'Variables\tMean \(sd\)', line) or \
           re.search(r'N \(baseline\)\t\d+\t\d+', line) or \
           re.search(table_pattern, line) or \
           re.search(r'Predictor\tcoef\tS\.E\tWald\tp', line) or \
           re.search(r'^\t\t.*?\t.*?\t.*?\t.*?\t.*?\t', line):
            continue
        cleaned_lines.append(line)
    text = '\n'.join(cleaned_lines)

    # 4. Remove reference IDs, statistical markers, and specific table explanations
    text = re.sub(r'\b\d{2}\.\d{4}\/\S+?\b', '', text) # DOI patterns
    text = re.sub(r'\(Fig\. \d+S?\)|\[\d+\]|\(Table \d+S?\)|\(see Fig\. \d+S?\)', '', text)
    text = re.sub(r'\s[X,x]2\s=\s[\d.]+', '', text)
    text = re.sub(r'\srho\s=\s\s?-\s?[\d.]+', '', text)
    text = re.sub(r'p\svalue\s=\s[\d.]+', '', text)
    text = re.sub(r'p\s=\s[\d.]+', '', text)
    text = re.sub(r'\(N\s=\s\d+\)', '', text)
    text = re.sub(r'\(Pearson\'s Chi-squared test; Chi2 = [\d.]+, p value = [\d.]+\)', '', text)
    text = re.sub(r'\(Pearson\'s Chi-squared test, Chi2 = [\d.]+, p value = [\d.]+\)', '', text)
    text = re.sub(r'\(Pearson\'s Chi-squared test, X2 = [\d.]+, p value = [\d.]+\)', '', text)
    text = re.sub(r'\(X2 = [\d.]+, p = <[\d.]+\)', '', text)
    text = re.sub(r'\(X2 = [\d.]+, p = [\d.]+\)', '', text)

    # Remove the list of abbreviations at the end of tables if they appear as prose
    text = re.sub(r'ALS, amyotrophic lateral sclerosis;.*?NCI, no cognitive impairment', '', text, flags=re.DOTALL)
    # Remove the "Coef represents..." explanation line for the table
    text = re.sub(r'Coef represents the value.*?in the model', '', text, flags=re.DOTALL)

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.strip("'")

    return text


In [ ]:
colnames=["year","id","title","journal_name","col5"]
df1=pd.read_csv(df1_dir,on_bad_lines='skip',names=colnames)
df1.head()

,year,id,title,journal_name,col5
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;"""
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;"""
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;"""
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;"""
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;"""


In [ ]:
df2=pd.read_csv(df2_dir,on_bad_lines='skip',names=colnames)
df2.head()

,year,id,title,journal_name,col5
0,2024,PMC10806872,Cytotoxic lesions of the corpus callosum (CLOC...,BMJ Neurol Open,"""Journal Article;"""
1,2024,PMC11192191,Childhood Cerebral Adrenoleukodystrophy: Case ...,Degener Neurol Neuromuscul Dis,"""Case Reports;Journal Article;"""
2,2024,PMC10968772,Effects of Cervical Spinal Manipulation on Sac...,Brain Sci,"""Case Reports;Journal Article;"""
3,2024,PMC11295153,A case of frontal lobe seizures with 'dancing-...,Eur J Neurol,"""Case Reports;Journal Article;"""
4,2024,PMC11022958,Cerebellar ependymal cyst: a case report.,Front Neurosci,"""Case Reports;Journal Article;"""


In [ ]:
df=pd.concat([df1,df2],ignore_index=True)
df.head()

,year,id,title,journal_name,col5
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;"""
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;"""
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;"""
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;"""
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;"""


In [ ]:
df['id'].count()

np.int64(9343)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/casereports')

from pmcfuncs import get_fulltext, get_fulltext_from_xml

In [ ]:
from pmcfuncs import get_abstract

In [ ]:
abs=get_abstract('PMC10864136')
print(abs)

<p>Differential diagnosis of focal brainstem lesions detected on MRI is challenging, especially in young children. Formerly, brainstem gliomas were classified mainly based on MRI features and location. However, since 2016, the World Health Organizationâs brainstem lesion classification requires tissue biopsy to reveal molecular characteristics. Although modern techniques of stereotactic or navigation-guided biopsy ensure accurate biopsy of the lesion with safety, biopsy of brainstem lesions is still generally not performed. Here, we report a focal brainstem lesion mimicking brainstem glioma in a 9-year-old girl. Initial MRI, MR spectroscopy, and <p><p>11<p><p>18<p>


In [ ]:
from tqdm import tqdm

abstracts = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    paper_id = str(row['id'])
    year = row['year']

    try:
        abstract = get_abstract(paper_id)
    except Exception as e:
        print(f"❌ Error for paper ID {paper_id}: {e}")
        abstract = ""  # Or you can use np.nan if you want missing values as NaN

    abstracts.append(abstract)

df['abstract'] = abstracts


  0%|          | 21/9343 [00:13<1:39:36,  1.56it/s]

❌ Error for paper ID PMC11408342: can only concatenate str (not "NoneType") to str


  0%|          | 24/9343 [00:15<1:38:38,  1.57it/s]

❌ Error for paper ID PMC11564264: can only concatenate str (not "NoneType") to str


  0%|          | 28/9343 [00:17<1:35:26,  1.63it/s]

...

100%|█████████▉| 9335/9343 [1:45:51<00:05,  1.53it/s]

❌ Error for paper ID PMC7974485: can only concatenate str (not "NoneType") to str


100%|█████████▉| 9337/9343 [1:45:53<00:03,  1.51it/s]

❌ Error for paper ID PMC8148741: can only concatenate str (not "NoneType") to str


100%|██████████| 9343/9343 [1:45:57<00:00,  1.47it/s]


In [ ]:
df['id'].isnull().any()

np.False_

In [ ]:
df.to_csv("data_with_abstracts.csv",index=False)

**Creating Topic Model**

In [29]:
#TOPIC MODEL
OUTPUT_DIR = "/content/drive/MyDrive/casereports/paper_embeddings_output"
EMBEDDINGS_FILE = os.path.join(OUTPUT_DIR, "specter_paper_embeddings.npy")
PAPER_IDS_FILE = os.path.join(OUTPUT_DIR, "specter_paper_ids.txt")
CLEAN_TEXT_FILE = os.path.join(OUTPUT_DIR, "specter_clean_texts.txt")

with open(PAPER_IDS_FILE, 'r', encoding='utf-8') as f:
    paper_ids = [line.strip() for line in f]

with open(CLEAN_TEXT_FILE, 'r', encoding='utf-8') as f:
    clean_texts = [line.strip() for line in f]

embeddings = np.load(EMBEDDINGS_FILE)

reducer = umap.UMAP(n_neighbors=10, n_components=2, metric='cosine', random_state=42)
clusterer = hdbscan.HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

topic_model = BERTopic(
    embedding_model=None,  # Use precomputed SPECTER embeddings
    vectorizer_model=CountVectorizer(stop_words="english"),
    umap_model=reducer,
    hdbscan_model=clusterer,
    verbose=True
)

topics, probs = topic_model.fit_transform(clean_texts, embeddings)

df = topic_model.get_document_info(clean_texts)
df["paper_id"] = paper_ids
df.to_csv(os.path.join(OUTPUT_DIR, "specter_bertopic_results.csv"), index=False)

topic_model.get_topic(-1)


2025-06-24 07:08:51,059 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-24 07:09:03,475 - BERTopic - Dimensionality - Completed ✓
2025-06-24 07:09:03,477 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-24 07:09:03,708 - BERTopic - Cluster - Completed ✓
2025-06-24 07:09:03,715 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-06-24 07:09:32,786 - BERTopic - Representation - Completed ✓


[('patients', 0.012197461452212085),
 ('patient', 0.011419969459429084),
 ('case', 0.010107651721237605),
 ('treatment', 0.00812917021092562),
 ('clinical', 0.008064484298990733),
 ('brain', 0.007735331548361471),
 ('symptoms', 0.0074781096065784395),
 ('disease', 0.007217406490709055),
 ('showed', 0.007205237348711993),
 ('reported', 0.006874684687478342)]

In [30]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2437,-1_patients_patient_case_treatment,"[patients, patient, case, treatment, clinical,...",[SARS-CoV-2-related encephalitis with prominen...
1,0,1352,0_patients_patient_case_clinical,"[patients, patient, case, clinical, treatment,...",[Case Report: Yellow Fever Vaccine-Associated ...
2,1,286,1_patients_patient_case_disease,"[patients, patient, case, disease, treatment, ...",[Cognitive impairment in multiple sclerosis: c...
3,2,273,2_patients_patient_case_disease,"[patients, patient, case, disease, brain, trea...",[Hyper Acute Demyelinating Encephalomyelitis o...
4,3,256,3_patients_patient_case_brain,"[patients, patient, case, brain, clinical, tre...",[Tractography Alterations in the Arcuate and U...
5,4,167,4_patients_patient_case_treatment,"[patients, patient, case, treatment, clinical,...",[Cognitive Sequelae of Central-Variant Posteri...
6,5,166,5_patient_patients_case_disease,"[patient, patients, case, disease, clinical, r...",[Primary neuroendocrine tumor of the pineal gl...
7,6,161,6_patients_patient_treatment_case,"[patients, patient, treatment, case, symptoms,...",[A case report of neuronal intranuclear inclus...
8,7,156,7_patient_patients_case_brain,"[patient, patients, case, brain, symptoms, syn...",[Acute Psychosis as Main Manifestation of Cent...
9,8,155,8_patients_patient_case_brain,"[patients, patient, case, brain, clinical, mri...",[Spontaneous Recovery from Unresponsive Wakefu...


**Visualizing Document Clusters**

In [31]:
# Get document info (includes topics, probabilities, etc.)
doc_info = topic_model.get_document_info(clean_texts)

# Add truncated text for hover display
doc_info["hover_text"] = doc_info["Document"].str.slice(0, 150) + "..."

# Add paper_id
doc_info["paper_id"] = paper_ids

# Get 2D UMAP embeddings from topic_model
umap_embeddings = topic_model.embedding_model.embedding_model if topic_model.embedding_model else None
umap_coords = topic_model.umap_model.transform(embeddings)

# Add UMAP coordinates
doc_info["umap_x"] = umap_coords[:, 0]
doc_info["umap_y"] = umap_coords[:, 1]

# Filter out outliers/noise (-1)
df_filtered = doc_info[doc_info['Topic'] != -1]

# Plot interactive 2D scatter plot
fig = px.scatter(
    df_filtered,
    x='umap_x',
    y='umap_y',
    color='Topic',
    hover_data={
        'Topic': True,
        'paper_id': True,
        'hover_text': True,
        'umap_x': False,
        'umap_y': False
    },
    title="BERTopic Document Clusters",
    width=1000,
    height=800
)

fig.update_layout(
    legend_title_text='Topic',
    legend=dict(itemsizing='constant'),
    hoverlabel=dict(bgcolor="white", font_size=12)
)

fig.show()


In [23]:
topic_model.save("specter_topic_model")

2025-06-24 06:59:21,795 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [15]:
doc_info = topic_model.get_document_info(clean_texts)
print(doc_info)

                                               Document  Topic  \
0     Excessively Delayed Radiation Changes After Pr...     13   
1     Excessively Delayed Radiation Changes After Pr...     -1   
2     Excessively Delayed Radiation Changes After Pr...      0   
3     Excessively Delayed Radiation Changes After Pr...      0   
4     Excessively Delayed Radiation Changes After Pr...     28   
...                                                 ...    ...   
7051  Diffusion-Weighted MRI in Recurrent Wernicke's...      7   
7052  Diffusion-Weighted MRI in Recurrent Wernicke's...     14   
7053  Diffusion-Weighted MRI in Recurrent Wernicke's...     -1   
7054  Diffusion-Weighted MRI in Recurrent Wernicke's...     -1   
7055  Diffusion-Weighted MRI in Recurrent Wernicke's...      7   

                                    Name  \
0            13_la_patient_case_patients   
1     -1_patients_patient_case_treatment   
2       0_patients_patient_case_clinical   
3       0_patients_patient_case

In [16]:
doc_topic_df = pd.DataFrame({
    "paper_id": paper_ids,
    "text": clean_texts,
    "topic": topics,
    "probability": probs
})


In [17]:
doc_topic_df.head()

,paper_id,text,topic,probability
0,PMC10899702,Excessively Delayed Radiation Changes After Pr...,13,0.978526
1,PMC10864136,Excessively Delayed Radiation Changes After Pr...,-1,0.000000
2,PMC10825037,Excessively Delayed Radiation Changes After Pr...,0,1.000000
3,PMC11063300,Excessively Delayed Radiation Changes After Pr...,0,1.000000
4,PMC10884197,Excessively Delayed Radiation Changes After Pr...,28,1.000000


In [19]:
doc_topic_df.size

28224

In [21]:
doc_topic_df[doc_topic_df['probability'] > 0.8]


,paper_id,text,topic,probability
0,PMC10899702,Excessively Delayed Radiation Changes After Pr...,13,0.978526
2,PMC10825037,Excessively Delayed Radiation Changes After Pr...,0,1.000000
3,PMC11063300,Excessively Delayed Radiation Changes After Pr...,0,1.000000
4,PMC10884197,Excessively Delayed Radiation Changes After Pr...,28,1.000000
6,PMC11521929,Excessively Delayed Radiation Changes After Pr...,13,1.000000
...,...,...,...,...
7046,PMC2854953,Diffusion-Weighted MRI in Recurrent Wernicke's...,14,0.956538
7048,PMC2671776,Diffusion-Weighted MRI in Recurrent Wernicke's...,12,1.000000
7050,PMC2854965,Diffusion-Weighted MRI in Recurrent Wernicke's...,0,0.996287
7051,PMC1592301,Diffusion-Weighted MRI in Recurrent Wernicke's...,7,0.935708
